In [40]:
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
import os
import tokenize
import string
from pickle import load
from keras.preprocessing.text import Tokenizer

In [ ]:
#load dataset and use features 
path="D:\Program Scripts\Machine Learning\datasets\Flicker8k_Dataset\Data"
def extract(path):
    model = VGG16()  #FROM https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
    model.layers.pop()#removes last added layer
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    features = dict()
    files=os.listdir(path)
    for i in range(0,len(files)-7):
        filename = path + '/' + files[i]
        image = load_img(filename, target_size=(224, 224))#Alex net Size
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = files[i].split('.')[0]
        features[image_id] = feature
    return features
features = extract(path)
print('Extracted Features: %d' % len(features))
dump(features, open('features.pkl', 'wb'))

        


In [41]:
#loading text data
def load_text(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
filename=path+"/Flickr8k.token.txt"
doc=load_text(filename)


In [42]:
#mapping image to multiple text description in train set using image id
def mapi(doc):
    mapping=dict()
    for i in doc.split("\n"):
        tok=i.split()
        if(len(tok)<2):
            continue
        image_id,image_des=tok[0],tok[1:]
        image_id = image_id.split('.')[0]
        image_des = ' '.join(image_des)
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_des)
    return mapping
description=mapi(doc)

In [43]:
#cleaning description remove puntuation and single letter char
def clean(description):
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in description.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            desc = desc.split()
            desc = [word.lower() for word in desc]
            desc = [w.translate(table) for w in desc]
            desc = [word for word in desc if len(word)>1]
            desc = [word for word in desc if word.isalpha()]
            desc_list[i] =  ' '.join(desc)

In [44]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(description):
    all_desc = set()
    for key in description.keys():
        [all_desc.update(d.split()) for d in description[key]]
    return all_desc
vocabulary = to_vocabulary(description)


In [45]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
save_descriptions(description, 'descriptions.txt')

In [46]:
#putting all data together
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    for line in doc.split('\n'):
        if len(line) < 1:
            continue
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)
#cleaned desc
def load_clean_descriptions(filename, dataset):
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]
        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = list()
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            descriptions[image_id].append(desc)
    return descriptions
#load image features
def load_image_features(filename,dataset):
    all_features = load(open(filename, 'rb'))
    features = {k: all_features[k] for k in dataset}
    return features
filename = path+'/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
#description
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# image features
train_features = load_image_features('features.pkl', train)
print('Photos: train=%d' % len(train_features))

Dataset: 6000
Descriptions: train=6000
Photos: train=6000


In [49]:
#conver dict to list desc
def to_lines(descriptions):
    all_desc=list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)


Vocabulary Size: 7378
